In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

%matplotlib inline

import pandas as pd
dataset = pd.read_csv('/kaggle/input/fffffff/features_dataset.csv')

2024-03-28 18:49:15.349782: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 18:49:15.349986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 18:49:15.526592: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
# Model 1: Feedforward Neural Network (FNN)
def create_feedforward_neural_network(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [3]:
!pip install pytorch-tabnet
!pip install python-whois
!pip install requests
!pip install dnspython
!pip install tldextract

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.0 MB/s eta 0:00:00


In [4]:
columns_of_interest = [
    "qty_slash_url",
    "time_domain_activation",
    "length_url",
    "directory_length",
    "file_length",
    "qty_dot_domain",
    "qty_at_params",
    "qty_dot_directory",
    "tls_ssl_certificate",
    "qty_mx_servers",
    "qty_at_url",
    "url_shortened",
    "qty_ip_resolved",
    "tld_present_params",
    "qty_nameservers",
    "domain_spf",
    "time_domain_expiration",
    "qty_hyphen_url",
    "qty_dot_file",
    "qty_hyphen_domain"
]
best_model='feed_forward'

In [5]:
def calculate_accur(predictions, actual_results):
    # Initialize a variable to count the number of correct predictions
    correct_predictions = 0

    # Iterate through each prediction and actual result
    for pred, actual in zip(predictions, actual_results):
        # Check if the prediction matches the actual result
        if pred == actual:
            # If so, increment the count of correct predictions
            correct_predictions += 1

    # Calculate accuracy by dividing the number of correct predictions by the total number of predictions
    accuracy = (correct_predictions / len(predictions)) * 100
    return accuracy


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

def preprocess_data(csv_path, columns_of_interest, test_size):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_path)

    # Replace -1 with mean values
    for col in df.columns:
        if -1 in df[col].unique():
            mean_value = int(df[col][df[col] != -1].mean())
            df[col] = df[col].replace(-1, mean_value)

    target_column = 'phishing'
    x = df.drop(target_column, axis=1)
    scaler = MinMaxScaler()

    # Apply Min-Max scaling to each column
    for col in x.columns:
        column_values = x[col].values.reshape(-1, 1)
        x[col] = scaler.fit_transform(column_values)

    df = pd.concat([x, df[target_column]], axis=1)

    X = df.drop('phishing', axis=1)
    y = df['phishing'].values

    selected_columns = X[columns_of_interest].values

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(selected_columns, y, test_size=test_size, random_state=42)

    return X_train, X_test, y_train, y_test



def train_and_test_model(X_train, y_train, X_test, y_test, epochs, input_dim):
    model = create_feedforward_neural_network(input_dim)
    model.fit(X_train, y_train, epochs=epochs, batch_size=64,verbose=0)
    #print(X_test.shape)
    y_pred = model.predict(X_test,verbose=0)
    threshold = 0.5

    # Calculate metrics
    y_pred_binary = (y_pred > threshold).astype(int)
    accuracy = accuracy_score(y_test, y_pred_binary)
    return accuracy, model

def predict_with_model(model, features, actual_results):
    features = features.reshape((features.shape[0], features.shape[2]))
    #print(features[0])
    y_pred_prob = model.predict(features,verbose=0)
    
    threshold = 0.5
    y_pred = (y_pred_prob > threshold).astype(int)

    accuracy = calculate_accur(y_pred, actual_results)
    return accuracy

def grid_hyperparameter_tuning(csv_paths, columns_of_interest_range, epochs_range, testing_sizes):
    results = []
    for csv_path in csv_paths:
        for columns_of_interest_number in range(columns_of_interest_range[0], columns_of_interest_range[1] + 1):
            for epochs in epochs_range:
                for test_size in testing_sizes:
                    X_train, X_test, y_train, y_test = preprocess_data(csv_path, columns_of_interest[:columns_of_interest_number], test_size)
                    accuracy, model = train_and_test_model(X_train, y_train, X_test, y_test, epochs, X_train.shape[1])


                    ds = pd.read_csv(csv_path)
                    V = ds[columns_of_interest[:columns_of_interest_number]]

                    mean_values = V.replace(-1, np.nan).mean(axis=0)
                    v_copy = dataset.drop('phishing', axis=1).replace(-1, np.nan)
                    v_copy.fillna(mean_values, inplace=True)
                    v_copy = v_copy.iloc[:, :columns_of_interest_number]
                    features = v_copy.values

                    min_values = V.replace(-1, np.nan).min(axis=0)
                    max_values = V.max(axis=0)
                    scaled_features = []

                    for row in features:
                        filtered_row = row[row != -1]
                        scaler = MinMaxScaler()
                        scaler.fit([min_values, max_values])
                        scaled_row = scaler.transform([filtered_row])
                        scaled_features.append(scaled_row)

                    scaled_example = np.array(scaled_features)

                    accuracy_on_new_data = predict_with_model(model,scaled_example, dataset['phishing'])

                    parameters = {
                        "csv_path": csv_path,
                        "columns_of_interest": columns_of_interest_number,
                        "epochs": epochs,
                        "test_size": test_size,
                        "accuracy_on_validation_data": accuracy,
                        "accuracy_on_new_data": accuracy_on_new_data
                    }
                    print(parameters)
                    results.append(parameters)
    return results

# Define parameters
csv_paths = ['/kaggle/input/zzzzzz/mendeley_dataset_full.csv',
             '/kaggle/input/ssssssss/dataset_small.csv']
columns_of_interest_range = [14, 20]
epochs_range = [10, 20, 30, 40, 50]
testing_sizes = [0.5, 0.4, 0.3, 0.2]

# Perform grid-based hyperparameter tuning
results = grid_hyperparameter_tuning(csv_paths, columns_of_interest_range, epochs_range, testing_sizes)
# Print results
for result in results:
    print(result)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


{'csv_path': '/kaggle/input/zzzzzz/mendeley_dataset_full.csv', 'columns_of_interest': 14, 'epochs': 10, 'test_size': 0.5, 'accuracy_on_validation_data': 0.9255256745781066, 'accuracy_on_new_data': 69.39632545931758}


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


{'csv_path': '/kaggle/input/zzzzzz/mendeley_dataset_full.csv', 'columns_of_interest': 14, 'epochs': 10, 'test_size': 0.4, 'accuracy_on_validation_data': 0.9285089822047999, 'accuracy_on_new_data': 71.31233595800525}


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

{'csv_path': '/kaggle/input/ssssssss/dataset_small.csv', 'columns_of_interest': 20, 'epochs': 50, 'test_size': 0.4, 'accuracy_on_validation_data': 0.9257822491260977, 'accuracy_on_new_data': 73.30708661417323}


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


{'csv_path': '/kaggle/input/ssssssss/dataset_small.csv', 'columns_of_interest': 20, 'epochs': 50, 'test_size': 0.3, 'accuracy_on_validation_data': 0.9307150164828919, 'accuracy_on_new_data': 73.35958005249343}


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


{'csv_path': '/kaggle/input/ssssssss/dataset_small.csv', 'columns_of_interest': 20, 'epochs': 50, 'test_size': 0.2, 'accuracy_on_validation_data': 0.9270185011509933, 'accuracy_on_new_data': 77.40157480314961}
{'csv_path': '/kaggle/input/zzzzzz/mendeley_dataset_full.csv', 'columns_of_interest': 14, 'epochs': 10, 'test_size': 0.5, 'accuracy_on_validation_data': 0.9255256745781066, 'accuracy_on_new_data': 69.39632545931758}
{'csv_path': '/kaggle/input/zzzzzz/mendeley_dataset_full.csv', 'columns_of_interest': 14, 'epochs': 10, 'test_size': 0.4, 'accuracy_on_validation_data': 0.9285089822047999, 'accuracy_on_new_data': 71.31233595800525}
{'csv_path': '/kaggle/input/zzzzzz/mendeley_dataset_full.csv', 'columns_of_interest': 14, 'epochs': 10, 'test_size': 0.3, 'accuracy_on_validation_data': 0.9305884564767813, 'accuracy_on_new_data': 73.07961504811898}
{'csv_path': '/kaggle/input/zzzzzz/mendeley_dataset_full.csv', 'columns_of_interest': 14, 'epochs': 10, 'test_size': 0.2, 'accuracy_on_validat

In [7]:
best_result = max(results, key=lambda x: x['accuracy_on_new_data'])

# Print the parameters of the best result
print("Best Result:")
for key, value in best_result.items():
    print(f"{key}: {value}")

Best Result:
csv_path: /kaggle/input/ssssssss/dataset_small.csv
columns_of_interest: 20
epochs: 40
test_size: 0.4
accuracy_on_validation_data: 0.9228834512746185
accuracy_on_new_data: 79.74628171478565
